In [1]:
%load_ext autoreload
%autoreload 2
from load_tether import *
import torch
import yaml
import logging
from imp import reload
import matplotlib.pyplot as plt
from liftpose.vision_3d import world_to_camera_dict, reprojection_error
reload(logging)
logger = logging.getLogger(__name__).setLevel(logging.INFO)
from tqdm import tqdm
tqdm.get_lock().locks = []

# decleare data parameters
par_train = {  'data_dir'       : '/data/LiftPose3D_2602/fly_tether/data_DF3D/', # change the path 
               'out_dir'        : './out',
               'train_subjects' : [1,2,3,4,5],
               'test_subjects'  : [6,7],
               'actions'        : ['all'],
               'cam_id'         : [1]}

# merge with training parameter
par_data = yaml.full_load(open('param.yaml', "rb"))
par = {**par_data["data"], **par_train}
plt.style.use('dark_background')

In [7]:
import torch
import pylab as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib
from matplotlib.animation import FFMpegWriter
from matplotlib.legend_handler import HandlerTuple
matplotlib.use('Agg')
import utils as utils
import transform as transform
import stats as stat
import plotting as plotting
from skeleton import skeleton
from tqdm import tqdm
import yaml
import sys
from load_tether import load_3D


cameras = [1,5]

#import
G, color_edge = skeleton() #skeleton
legtips = [4, 9, 14, 19, 24, 29]


from liftpose.main import test
#load data / statistics for cameras
out, tar = [], []
for cam in cameras:
    data_dir = f'out_drop_cam[{cam}]'
    
    test_3d, _, rcams_test = load_3D(
        par["data_dir"],
        par,
        cam_id=[cam],
        subjects=par["test_subjects"],
        actions=par["actions"],
    )
    tar_mean = torch.load(os.path.join(par["data_dir"], 'stat_3d.pth.tar'))['mean']
    tar_std = torch.load(os.path.join(par["data_dir"], 'stat_3d.pth.tar'))['std']
    targets_3d = torch.load(os.path.join(par["data_dir"], 'stat_3d.pth.tar'))['targets_3d']
    offset = torch.load(os.path.join(par["data_dir"], 'stat_3d.pth.tar'))['offset']
    offset = np.concatenate([v for k,v in offset.items()], 0)
    
    #load predictions
    data = torch.load(os.path.join(par["data_dir"], '/test_results.pth.tar'))
    tar_ = data['target']
    out_ = data['output']
    
    #expand
    tar_ = utils.add_roots(tar_, targets_3d, len(tar_mean))
    out_ = utils.add_roots(out_, targets_3d, len(tar_mean))
    
    #unnormalise
    tar_ = stat.unNormalize(tar_, tar_mean, tar_std) 
    out_ = stat.unNormalize(out_, tar_mean, tar_std)
    
    #translate legs back to their original places
    tar_ += offset[0,:]
    out_ += offset[0,:]
    
    #transform back to worldcamera_to_world( poses_cam, cam_par, cam )
    tar_ = transform.camera_to_world(tar_, list(rcams_test.values())[11])
    out_ = transform.camera_to_world(out_, list(rcams_test.values())[11])
    
    #tar_ = utils.filter_data(tar_, window=5, order=2)
    #out_ = utils.filter_data(out_, window=5, order=2)
    
    tar.append(tar_)
    out.append(out_)

#combine cameras
#tar = average_cameras(tar)
#out = average_cameras(out)
tar = np.concatenate(tar, axis=1)
out = np.concatenate(out, axis=1)

# Set up a figure
fig = plt.figure(figsize=plt.figaspect(1))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
ax = fig.add_subplot(111, projection='3d')
#ax.view_init(elev=30, azim=25)

writer = FFMpegWriter(fps=10)
with writer.saving(fig, "LiftPose3D_prediction_fly_tether_MDN.mp4", 100):
    for t in tqdm(range(55838, 55838 + 900)):
        pos_pred, pos_tar = [], []
        
        ax.cla()
        
        thist = 7
        pos_pred, pos_tar = [], []
        for j in range(out.shape[1]//3):
            tmin = max(0,t-thist+1)
            pos_pred.append((out[tmin:(t+1), 3*j], out[tmin:(t+1), 3*j+1], out[tmin:(t+1), 3*j+2]))
            pos_tar.append((tar[tmin:(t+1), 3*j], tar[tmin:(t+1), 3*j+1], tar[tmin:(t+1), 3*j+2]))
        pos_pred, pos_tar = np.array(pos_pred), np.array(pos_tar)

        pos_tar = rotate_points3d(pos_tar.transpose(2,0,1)).transpose(1,2,0)
        pos_pred = rotate_points3d(pos_pred.transpose(2,0,1)).transpose(1,2,0)

        plotting.plot_trailing_points(pos_pred[legtips,:,:],min(thist,t+1),ax)
        
        #plot skeleton
        plotting.plot_3d_graph(G, pos_tar[:,:,-1], ax, color_edge=color_edge)    
        plotting.plot_3d_graph(G, pos_pred[:,:,-1], ax, color_edge=color_edge, style='--')
            
        #### this bit is just to make special legend 
        pts = np.array([1,1])
        p1, = ax.plot(pts, pts, pts, 'r-')
        p2, = ax.plot(pts, pts, pts, 'b-')
        p3, = ax.plot(pts, pts, pts, 'r--', dashes=(2, 2))
        p4, = ax.plot(pts, pts, pts, 'b--', dashes=(2, 2))
        ax.legend([(p1, p2), (p3, p4)], 
            ['Triangulated 3D pose using 3 cameras per keypoint', \
             'LiftPose3D prediction using 1 camera per keypoint'], 
            numpoints=1, handler_map={tuple: HandlerTuple(ndivide=None)},
            loc=(0.1,0.9),
            frameon=False)    
        p1.remove()
        p2.remove()
        p3.remove()
        p4.remove()
        ####    
           
        #ax.set_xlim([-2,5])
        #ax.set_ylim([-1,4])
        #ax.set_zlim([1,2.75])
        
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_zticklabels([])
        ax.grid(True)
        
        writer.grab_frame()

100%|██████████| 900/900 [02:00<00:00,  7.45it/s]


In [62]:
!ffmpeg -i DeepFly3D_prediction_fly_tether_MDN.mp4 -i LiftPose3D_prediction_fly_tether_MDN.mp4 -filter_complex "[0:v]scale=-1:400[v0];[v0][1:v]hstack=inputs=2" Video_1.mp4

ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9.3.0 (crosstool-NG 1.24.0.133_b0863d8_dirty)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1609680890771/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1609680890771/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/home/conda/feedstock_root/build_artifacts/ffmpeg_1609680890771/_build_env/bin/pkg-config
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavforma